# Subset scenes, DEMs

In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import shapely
import geopandas as gpd
import rasterio as rio
from rasterio.windows import Window
import rasterio.mask 
import shutil
import glob
from osgeo import gdal
import xarray as xr
from PIL import Image
import time
from scipy import ndimage

In [2]:
# set paths
loc = 'frontrange'
track = 'des'

desktop = '/Users/qbren/Desktop'
main_dir = f'{desktop}/taco/projects/atmospheric_correction/proc/training_data'
igram_dir = f'{main_dir}/{track}_crop_{loc}_noise_era5'
subsets_dir = f'subsets_{loc}_{track}'
veloc_fn = f'{main_dir}/veloc_crop/{track}_veloc_{loc}.tif'

In [3]:
def clump_size(array, quantile_threshold=0.99, return_mask=False):
    quantile = np.quantile(np.absolute(array), q=quantile_threshold) # find threshold based on quantile
    mask = np.where(np.absolute(array)>quantile, 1, 0) # get areas above threshold
    s = [[1,1,1],[1,1,1],[1,1,1]] # structure for clumping kernel 
    features, count = ndimage.label(mask, s) # clump features
    feature_ids, feature_counts = np.unique(features, return_counts=True) # count pixels in features
    feature_counts = np.delete(feature_counts, feature_counts.argmax()) # throw away background
    
    if return_mask==True:
        return mask
    
    return feature_counts.mean()
    
# I think the appropriate mean for keeping a subset is 3 pixels per clump with a quantile threshold of 0.99

In [ ]:
%%time

# load, crop, and save rasters

# list interferograms
dirs = os.listdir(igram_dir)
subsets_total = 0
subsets_desired = 1500
igram_loop = 0

while subsets_total < subsets_desired: # loop until number of desired subsets is satisfied
    igram_loop += 1

    # loop through interferograms
    for i, idir in enumerate(dirs):
        if i > 0:
            subsets_total += subsets
            print(f'total subsets: {subsets}, total counter: {counter}')
        cwd = f'{igram_dir}/{idir}'
        os.chdir(cwd)

        # select the line of sight displacement
        allfiles = os.listdir(cwd)
        for fn in allfiles:
            if fn[-12:] == 'los_disp.tif':
                noise_tif = fn
            elif fn[-7:] == 'dem.tif':
                dem_tif = fn
            elif fn[-8:] == 'ERA5.tif':
                era5_tif = fn
        noise_tif_fn = f'{igram_dir}/{idir}/{noise_tif}'
        dem_tif_fn =  f'{igram_dir}/{idir}/{dem_tif}'
        era5_tif_fn = f'{igram_dir}/{idir}/{era5_tif}'
        xsize, ysize = 125, 125  # define the test image size
        print("working on: ", idir)

        subsets = 0 # reset subset number
        counter = 0 # reset counter 
        timeout = time.time() + 60*3 # set time to spend on each igram

        # loop until subsetting is finished
        while time.time() < timeout:
            counter += 1

            # open src image
            noise_src = rio.open(noise_tif_fn)
            noise_rs = noise_src.read(1) # also open as np array

            dem_src = rio.open(dem_tif_fn)
            dem_rs = dem_src.read(1) # also open as np array

            veloc_src = rio.open(veloc_fn)
            veloc_rs = veloc_src.read(1) # also open as np array
            
            era5_src = rio.open(era5_tif_fn)
            era5_rs = era5_src.read(1)

            # create bounds for new origin location
            xmin, xmax = 0, noise_src.width - xsize
            ymin, ymax = 0, noise_src.height - ysize
            xoff, yoff = np.random.randint(xmin, xmax), np.random.randint(ymin, ymax)

            temp_noise = noise_rs[yoff:(yoff+ysize), xoff:(xoff+xsize)]
            temp_dem = dem_rs[yoff:(yoff+ysize), xoff:(xoff+xsize)]
            temp_veloc = veloc_rs[yoff:(yoff+ysize), xoff:(xoff+xsize)]
            temp_era5 = era5_rs[yoff:(yoff+ysize), xoff:(xoff+xsize)]

            if not 0.0 in temp_noise: # only save if there are no nodata values in subset
                if np.median(temp_dem) >= 3300: # only save if subset is at high elevation
                    if clump_size(temp_veloc) >= 3.0: # only save subsets with big moving features
                        window = Window(xoff, yoff, xsize, ysize)
                        noise_transform = noise_src.window_transform(window)
                        dem_transform = dem_src.window_transform(window)
                        veloc_transform = veloc_src.window_transform(window)
                        era5_transform = era5_src.window_transform(window)

                        # write subset
                        noise_profile = noise_src.profile
                        dem_profile = dem_src.profile
                        veloc_profile = veloc_src.profile
                        era5_profile = era5_src.profile

                        noise_profile.update({
                            'height': xsize,
                            'width': ysize,
                            'transform': noise_transform})
                        dem_profile.update({
                            'height': xsize,
                            'width': ysize,
                            'transform': dem_transform})
                        veloc_profile.update({
                            'height': xsize,
                            'width': ysize,
                            'transform': veloc_transform})
                        era5_profile.update({
                            'height': xsize,
                            'width': ysize,
                            'transform': era5_transform})

                        noise_out_fn = f'{desktop}/{subsets_dir}/int/{track}{igram_loop}.{subsets}_{noise_tif[0:-13]}.tif'
                        dem_out_fn = f'{desktop}/{subsets_dir}/dem/{track}{igram_loop}.{subsets}_{noise_tif[0:-13]}.tif'
                        veloc_out_fn = f'{desktop}/{subsets_dir}/veloc/{track}{igram_loop}.{subsets}_{noise_tif[0:-13]}.tif'
                        era5_out_fn = f'{desktop}/{subsets_dir}/era5/{track}{igram_loop}.{subsets}_{noise_tif[0:-13]}.tif'

                        with rasterio.open(noise_out_fn, 'w', **noise_profile) as noise_dst:
                            # Read the data from the window and write it to the output raster
                            noise_dst.write(noise_src.read(window=window))
                        noise_src.close()

                        with rasterio.open(dem_out_fn, 'w', **dem_profile) as dem_dst:
                            # Read the data from the window and write it to the output raster
                            dem_dst.write(dem_src.read(window=window))
                        dem_src.close()

                        with rasterio.open(veloc_out_fn, 'w', **veloc_profile) as veloc_dst:
                            # Read the data from the window and write it to the output raster
                            veloc_dst.write(veloc_src.read(window=window))
                        veloc_src.close()
                        
                        with rasterio.open(era5_out_fn, 'w', **era5_profile) as era5_dst:
                            # Read the data from the window and write it to the output raster
                            era5_dst.write(era5_src.read(window=window))
                        era5_src.close()

                        subsets+=1 # update index

working on:  S1AA_20180626T130936_20180708T130937_VVP012_INT40_G_ueF_E4CA
total subsets: 113, total counter: 935
working on:  S1AA_20180708T130937_20180720T130938_VVP012_INT40_G_ueF_866A
total subsets: 116, total counter: 948
working on:  S1AA_20180720T130938_20180801T130938_VVP012_INT40_G_ueF_2D55
total subsets: 119, total counter: 954
working on:  S1AA_20180801T130938_20180813T130944_VVP012_INT40_G_ueF_DF60
total subsets: 129, total counter: 952
working on:  S1AA_20180813T130944_20180825T130944_VVP012_INT40_G_ueF_F926
total subsets: 125, total counter: 952
working on:  S1AA_20180825T130944_20180906T130945_VVP012_INT40_G_ueF_1556
total subsets: 90, total counter: 958
working on:  S1AA_20180906T130945_20180918T130945_VVP012_INT40_G_ueF_F15C
total subsets: 48, total counter: 976
working on:  S1AA_20180918T130945_20180930T130946_VVP012_INT40_G_ueF_3CC2
total subsets: 123, total counter: 952
working on:  S1AA_20180930T130946_20181012T130946_VVP012_INT40_G_ueF_99C9
total subsets: 0, total 

## Interpolate gaps if need be

In [ ]:
# list subsets 
noise_fns = os.listdir(f'{desktop}/{subsets_dir}/int')

def list_tifs(my_fns):
    my_list = []
    for i in my_fns:
        if i[-4:] == '.tif':
            my_list.append(i)
    return my_list

noise_list = list_tifs(noise_fns)

In [ ]:
zeros=[]

for i, fn in enumerate(noise_list):
        noise_fn = f'{desktop}/{subsets_dir}/int/{fn}'

        noise_src = rio.open(noise_fn)
        noise = noise_src.read(1) # also open as np array
        zeros.append(np.count_nonzero(noise==0))

In [ ]:
plt.hist(zeros, bins=300);

In [ ]:
# interpolate holes
#for i, fn in enumerate(noise_list):
#        noise_fn = f'{desktop}/{subsets_dir}/int/{fn}'
#
#        noise_src = rio.open(noise_fn)
#        noise = noise_src.read(1) # also open as np array
#        noise = xr.DataArray(noise).interpolate_na(dim='dim_0', method='linear').fillna(value=0).values
#
#        out_path = f'{desktop}/{subsets_dir}/int_interp/{fn}'

#        im = Image.fromarray(noise)
#        im.save(out_path)